# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [ ]:
import altair as alt
import pandas as pd
import re
alt.data_transformers.enable("vegafusion")

## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [100]:
legdf = pd.read_csv("data/legislators.csv")


In [ ]:
def legislators_df():
    '''
    Creates a state column by removing the extraneous text in jurisdiction from each side
    Returns the data as a dataframe with the state column included
    '''
    legdf = pd.read_csv("data/legislators.csv")
    state_series = legdf["jurisdiction"]
    front_removed = state_series.str.split(pat=":", expand=True)
    back_removed = front_removed[2].str.split(pat="/", expand=True)
    legdf["state"] = back_removed[0]
    return legdf

# render dataframe 
legislators_df()

### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [ ]:
def states_by_gender_initial(df):
    '''
    Input: Pandas Dataframe
    Output: Basic Bar Chart for gender of legislators by state
    '''
    initial = alt.Chart(df).mark_bar().encode(
        alt.X('state'),
        alt.Y('count()'),
        alt.Color('gender')
    )
    return initial

# render chart
states_by_gender_initial(legislators_df())

### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: <https://blog.datawrapper.de/gendercolor/>)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png*.

In [ ]:
# Exact Colors for Gender:
# Male, Female, Other
gender_colors = ['#1fc3aa','#8624f5', 'orange']

def states_by_gender_improved(df):
    '''
    Input: Pandas Dataframe
    Normalizes gender counts in each legislature, gives them non-political colors, and adds a line at 50% for parity
    Output: Bar Chart object of normalized gender of legislators by state
    '''
    bar = alt.Chart(df).mark_bar().encode(
        x = 'state',
        y = alt.Y('count()').stack("normalize"),
        color = alt.Color('gender', scale=alt.Scale(range=gender_colors))
    )
    parity_line = alt.Chart(df).mark_rule().encode(
        y= alt.datum(.5)
    )

    return bar + parity_line

# render chart
states_by_gender_improved(legislators_df())

## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [ ]:
def party_control_raw(df):
    '''
    Input: Data as a pandas dataframe
    Output: Normalized Bar Chart of party control of legislature by state
    '''
    # Base Normalized Bars, sorted by party
    bar = alt.Chart(df).mark_bar().encode(
        x = 'state',
        y = alt.Y('count()').stack("normalize"),
        color = alt.Color('party')
    )
    # Line at 50% to mark control of legislature
    parity_line = alt.Chart(df).mark_rule().encode(
        y= alt.datum(.5)
    )

    return bar + parity_line

party_control_raw(legislators_df())

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [ ]:
def clean_party_df():
    # start with the DataFrame from part 1 & return transformed copy
    df = legislators_df()

    #Drop States with uncooperative parties
    df = df[df["state"] != "dc"]
    df = df[df["state"] != "pr"]
    df = df[df["state"] != "ne"]

    #Create a party code to aggregate party names
    for ind, party in df["party"].items():
        if re.search('Democratic', party):
            df.loc[ind, "party_code"] = "D"
        elif re.search('Republican', party):
            df.loc[ind, "party_code"] = "R"
        else:
            df.loc[ind, "party_code"] ="O"
    return df

clean_party_df()


### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.

In [ ]:
party_colors = ["#34b4eb", "#0f990c", "#800427"]

def party_control_by_chamber(df):

    bar = alt.Chart().mark_bar().encode(
        x = alt.Y('count()').stack("normalize"),
        y = 'state',
        color = alt.Color('party_code', scale=alt.Scale(range=party_colors))
    )
    parity_line = alt.Chart().mark_rule().encode(
        x = alt.datum(.5)
        )
    
    # Splits the chart by legislative chamber
    final_chart = alt.layer(bar, parity_line, data = df).facet("type")

    return final_chart

party_control_by_chamber(clean_party_df())

## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [107]:
def population_combined_df():
    legdf = legislators_df()
    popdf = pd.read_csv("data/populations.csv")

    lowerdf = legdf[legdf["type"] == "lower"]
    upperdf = legdf[legdf["type"] == "upper"]

    for ind, state in popdf["state"].items():
        popdf.loc[ind, "upper"] = upperdf[upperdf["state"] == state].shape[0]
        popdf.loc[ind, "lower"] = lowerdf[lowerdf["state"] == state].shape[0]

    return popdf

#population_combined_df()

### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [ ]:
def scatter_pop_size(df):
    '''
    Input: Pandas Dataframe
    Output: Scatter Plot Chart of Legislature Size vs State Population, with adjusted points
    for upper and lower houses.
    '''
    #Title and Axis Name adjustments made in primary chart
    lower_point = alt.Chart(df, title = "Legislature Size vs State Population").mark_point(
        color='orange',
        shape = "triangle-down"
    ).encode(
        alt.X("pop_2020").title("State Population (2020)"),
        alt.Y("lower").title("Legislature Size"),   
    )
    upper_point = alt.Chart(df).mark_point(
        color='purple',
        shape = "triangle-up"
    ).encode(
        x = "pop_2020",
        y = "upper",
    )
    
    final_chart = alt.layer(upper_point, lower_point, data = df)
    return final_chart


scatter_pop_size(population_combined_df())

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [ ]:
def scatter_pop_size_regression(df):
    '''
    Input: Pandas Dataframe
    Output: Scatter Plot Chart of Legislature Size vs State Population, with adjusted points
    for upper and lower houses, with added trend lines.
    '''
    lower_point = alt.Chart(df, title = "Legislature Size vs State Population").mark_point(
        color='orange',
        shape = "triangle-down"
    ).encode(
        alt.X("pop_2020").title("State Population (2020)"),
        alt.Y("lower").title("Legislature Size")
    )
    upper_point = alt.Chart(df).mark_point(
        color='purple',
        shape = "triangle-up"
    ).encode(
        x = "pop_2020",
        y = "upper",
    )
    upper_line = upper_point.transform_regression("pop_2020","upper").mark_line(
        color = "purple"
    )
    lower_line = lower_point.transform_regression("pop_2020","lower").mark_line(
        color = "orange"
    )

    final_chart = alt.layer(upper_line, lower_line, upper_point, lower_point, data = df)
    return final_chart

scatter_pop_size_regression(population_combined_df())

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [110]:
# Note: I appear to be incapable of clearing out the unnecessary time data, which appears
# to be the minor issue faced in chart 4.3. Attempted reading in dates directly in read_csv (failed)
# as well as trying to .dt.floor them (failed due to datetime objects... not being datetime objects)

def actions_df():
    actiondf = pd.read_csv("data/actions_il-in-mi-wi_2021-2024.csv")
    actiondf["date"] = pd.to_datetime(actiondf["date"], yearfirst = True, format='mixed')
    return actiondf

### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [ ]:
def actions_heatmap_scaled(df):
    heat = alt.Chart(df).mark_rect().encode(
        x = 'yearweek(date)',
        y = 'state',
        color = 'count()'
    ).properties(
    width=900,
    height=150
)
    return heat

actions_heatmap_scaled(actions_df())

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [ ]:
base_data = actions_df()
exclude_il = base_data[base_data["state"] != "IL"]

actions_heatmap_scaled(exclude_il)

In [ ]:
only_il = base_data[base_data["state"] == "IL"]

actions_heatmap_scaled(only_il)

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [ ]:
def actions_cumulative(df):
    '''
    Input: Pandas Dataframe
    Output: Chart showing cumulative legislative actions taken by state over the course of 4 years.
    '''
    cumulative = alt.Chart(df, width = 800).transform_window(
        # Split the cumulative counts by state
        groupby = ["state"],
        sort=[{'field': 'date'}],
        # Creates a sum of counts for each cumulative line
        total_actions='count()'
    ).mark_line().encode(
        x ='yearweek(date)',
        y = "total_actions:Q",
        color = "state:N"
    )
    return cumulative

# Workaround for actions_cumulative getting fussy about being passed a function instead of a dataframe.
x = actions_df()
actions_cumulative(x)